# AutoSFX

In [1]:
import importlib
import classify
import scene_understanding
import sync
from PIL import Image
import openai
import os
from openai import OpenAI
import cv2

## Classify

In [22]:
importlib.reload(scene_understanding)

In [42]:
importlib.reload(classify)

<module 'classify' from '/home/s5614279/Master Project/AutoSFX/classify.py'>

In [1]:
video_path = "/home/s5614279/Master Project/running.mp4"  # your video path

In [ ]:
# create SceneUnderstanding object
scene_understanding = scene_understanding.SceneUnderstanding()


resized_frame = scene_understanding.process_video(video_path)
#frame_tags = []
#frame_context = []

frame_values = []
frame_objects = []

#output_dir = 'output_frames'
#os.makedirs(output_dir, exist_ok=True)

#for i, frame in enumerate(resized_frame[::15]):
for i, frame in enumerate(resized_frame[::5]):
    pil_image = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    #tags, context = scene_understanding.analyze_image(pil_image)
    #frame_tags.append(tags)
    #frame_context.append(context)

    values, objects = scene_understanding.analyze_image(pil_image)
    frame_score = {

    }
    frame_values.append(values)
    frame_objects.append(objects)

    #image_save_path = os.path.join(output_dir, f'frame_{i:04d}.jpg')
    #pil_image.save(image_save_path)


client = OpenAI()

api_key = os.getenv('OPENAI_API_KEY')

openai.api_key = api_key

completion = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": "You will be provided with a context describing a scene, and your task is to give the sound suggestions that collectively form the audio landscape of the described scene"},
    {"role": "user", "content": f"What do I hear in the following video description, give me a list: {frame_context[0]}"}
  ]
)

print(completion.choices[0].message)


In [6]:
classify = classify.Classify()

resized_frame = classify.process_video(video_path)

frame_values = []
frame_objects = []
frame_scores = []

for i, frame in enumerate(resized_frame[::2]):
    pil_image = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

    values, objects = classify.recognize_objects(pil_image)
    frame_score = {
        'object': objects, #top 5 
        'score': values
    }
    frame_values.append(values)
    frame_objects.append(objects)
    frame_scores.append(frame_score)


Now you can see the interfer result of every frame.

In [7]:
print(f"Object in the scene:  {frame_objects[103][0]}  {frame_values[103][0]}")

Object in the scene:  Footsteps walking running  0.4453125


## Sync

In [3]:
syncer = sync.ObjectIntervalSync(video_path)
syncer.analyze_frames()
syncer.calculate_intervals()
intervals = syncer.get_intervals()
print(intervals)

{'Dog': [], 'Rain': [], 'Crying baby': [], 'Door knock': [], 'Helicopter': [], 'Horse': [], 'Rooster': [], 'Sea waves': [], 'Sneezing': [], 'Mouse click': [], 'Chainsaw': [], 'Pig': [], 'Crackling fire': [], 'Clapping': [], 'Keyboard typing': [], 'Siren': [], 'Cow': [], 'Crickets': [], 'Breathing': [], 'Door, wood creaks': [], 'Car horn': [], 'Frog': [], 'Chirping birds': [], 'Coughing': [], 'Can opening': [], 'Engine': [], 'Cat': [], 'Water drops': [], 'Footsteps walking running': [(0, 115, 3.8333333333333335), (117, 142, 0.8333333333333334)], 'Washing machine': [], 'Train': [], 'Hen': [], 'Wind': [], 'Laughing': [], 'Vacuum cleaner': [], 'Church bells': [], 'Insects (flying)': [], 'Pouring water': [], 'Brushing teeth': [], 'Clock alarm': [], 'Airplane': [], 'Sheep': [], 'Toilet flush': [], 'Snoring': [], 'Clock tick': [], 'Fireworks': [], 'Crow': [], 'Thunderstorm': [], 'Drinking, sipping': [], 'Glass breaking': [], 'Hand saw': []}


We have the intervals, and then we can retrieve the sound effects audio based on the intervals.

In [5]:
import retrieve
import IPython.display as ipd
import librosa

video_path = "/home/s5614279/Master Project/running.mp4" 
csv_path = "/home/s5614279/Master Project/AutoSFX/data/meta/esc50.csv" 
audio_folder = "/home/s5614279/Master Project/AutoSFX/data/audio/" 

matched_audios = retrieve.retrieve_audio(video_path, csv_path, audio_folder)

for obj, audios in matched_audios.items():
    for audio_info in audios:
        print(f"Object: {obj}, Interval: {audio_info['interval']}, Audio File: {audio_info['audio_file']}")
        
        # 使用librosa加载音频
        audio_data, sr = librosa.load(audio_info['audio_file'], sr=None)
        
        # 使用IPython.display.Audio播放音频
        display(ipd.Audio(data=audio_data, rate=sr))

## Retrieve

In [ ]:
import retrieve
from IPython.display import Audio, display

video_path = "/path/to/your/video.mp4" 
csv_path = "/mnt/data/esc50.csv" 
audio_folder = "/path/to/audio/folder" 

# 调用retrieve_audio函数
matched_audios = retrieve.retrieve_audio(video_path, csv_path, audio_folder)

for obj, audios in matched_audios.items():
    for audio_info in audios:
        print(f"Object: {obj}, Interval: {audio_info['interval']}, Audio File: {audio_info['audio_file']}")
        display(Audio(audio_info['audio_file']))